In [1]:
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm,trange
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
import torch
from torchvision.transforms import ToTensor
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import confusion_matrix
import time
from PIL import Image,ImageEnhance
import tensorflow as tf
from itertools import islice
from efficientnet_pytorch import EfficientNet



os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



import warnings
warnings.filterwarnings("ignore")


Using TensorFlow backend.


In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [5]:
max_frames = 10
max_df = 2
window_size= 5
supersteps = (max_frames-window_size+1)

df_train0 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_0/metadata.json')
df_train1 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_1/metadata.json')
df_train2 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_2/metadata.json')
df_train3 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_3/metadata.json')
df_train4 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_4/metadata.json')
df_train5 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_5/metadata.json')
df_train6 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_6/metadata.json')
df_train7 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_7/metadata.json')
df_train8 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_8/metadata.json')
df_train9 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_9/metadata.json')
df_train10 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_10/metadata.json')
df_train11 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_11/metadata.json')
df_train12 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_12/metadata.json')
df_train13 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_13/metadata.json')
df_train14 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_14/metadata.json')
df_train15 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_15/metadata.json')
df_train16 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_16/metadata.json')
df_train17 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_17/metadata.json')
df_train18 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_18/metadata.json')
df_train19 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_19/metadata.json')
df_train20 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_20/metadata.json')
df_train21 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_21/metadata.json')
df_train22 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_22/metadata.json')
df_train23 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_23/metadata.json')
df_train24 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_24/metadata.json')
df_train25 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_25/metadata.json')
df_train26 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_26/metadata.json')
df_train27 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_27/metadata.json')
df_train28 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_28/metadata.json')
df_train29 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_29/metadata.json')
df_train30 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_30/metadata.json')
df_train31 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_31/metadata.json')
df_train32 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_32/metadata.json')
df_train33 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_33/metadata.json')
df_train34 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_34/metadata.json')
df_train35 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_35/metadata.json')
df_train36 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_36/metadata.json')
df_train37 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_37/metadata.json')
df_train38 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_38/metadata.json')
df_train39 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_39/metadata.json')
df_train40 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_40/metadata.json')
df_train41 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_41/metadata.json')
df_train42 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_42/metadata.json')
df_train43 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_43/metadata.json')
df_train44 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_44/metadata.json')
# df_train45 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_45/metadata.json')





df_train47 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_47/metadata.json')
df_train48 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_48/metadata.json')
df_train49 = pd.read_json('/home/aelbakry1999/dfdc/dfdc_train_part_49/metadata.json')






df_train_all = [df_train0, df_train1, df_train2, df_train3, df_train4,df_train5, df_train6, df_train7, df_train8, df_train9, 
                df_train10, df_train11, df_train12, df_train13, df_train14, df_train15, df_train16, df_train17, df_train18,
                df_train19, df_train20, df_train21, df_train22, df_train23, df_train24, df_train25, df_train26, df_train27,
                df_train28, df_train29, df_train30, df_train31, df_train32, df_train33, df_train34, df_train35, df_train36, 
                df_train37, df_train38, df_train39, df_train40, df_train41, df_train42, df_train43, df_train44]

df_test_all = [df_train47, df_train48, df_train49]


In [6]:
LABELS = ['REAL','FAKE']
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
efficientnet = EfficientNet.from_pretrained('efficientnet-b6').to(device)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /home/aelbakry1999/.cache/torch/checkpoints/efficientnet-b6-c76e70fd.pth



Loaded pretrained weights for efficientnet-b6


In [39]:
"""function to embed video frames with InceptionResnetV1"""
def embed(frames):
    faces_embedded = []
    tf_img = lambda i: ToTensor()(i).unsqueeze(0)
    embeddings = lambda input: efficientnet.extract_features(input)

    with torch.no_grad():
        for faces in tqdm(frames):
            vid_embs = []
            for i in range(max_frames):
                t = tf_img(faces[i]).to(device)
                e = embeddings(t).squeeze().cpu().tolist()
                vid_embs.append(e)
            faces_embedded.append(vid_embs)

    return  faces_embedded


In [8]:
"""function to read video frames from given paths"""
def read_img(path):
    frames = []
    for i in range(max_frames):
        frames.append(cv2.cvtColor(cv2.imread(path[i]),cv2.COLOR_BGR2RGB))
    return frames


In [9]:
"""function to flip video frames from given paths for data Augmentation """
def flip_horizontal(path):
    frames_flipped = []
    for i in range(max_frames):
        frames_flipped.append(cv2.flip(cv2.cvtColor(cv2.imread(path[i]),cv2.COLOR_BGR2RGB), 1))
    return frames_flipped

def saturation(path):
    factor = 3
    frames_saturation = []
    for i in range(max_frames):
        frames_saturation.append(ImageEnhance.Color(Image.open(path[i])).enhance(factor))
    return frames_saturation

def contrast(path):
    factor = 3
    frames_contrast = []
    for i in range(max_frames):
        frames_contrast.append(ImageEnhance.Contrast(Image.open(path[i])).enhance(factor))
    return frames_contrast


In [10]:
def load_data(index, df_train):
    paths=[]
    y=[]

    df_train_values = list(df_train.columns.values)


    for value in df_train_values:
        image_paths=[]

        try:
            for num in range(max_frames):
                path = '/home/aelbakry1999/images/margin_0/dfdc_train_part_' + str(index) +"/"+ value.replace('.mp4', '') + '/frame' + str(num) +'.jpeg'
                image_paths.append(path)
                if not os.path.exists(path):
                    # print(path)
                    raise Exception
                
            paths.append(image_paths)
            y.append(LABELS.index(df_train[value]['label']))

        except Exception as err:
                # print(err)
            pass


    return paths, y


In [11]:
# all_paths=[]
# all_y=[]

# """Loading all paths and y_labels in df_train_all """
# # print("Loading training paths and y values from JSON files")
# for index in tqdm(range(np.shape(df_train_all)[0])):
#     path, labels = load_data(index, df_train_all[index])
#     all_paths.extend(path)
#     all_y.extend(labels)

# print(len(all_paths) )
# print(len(all_y) )


In [12]:
# paths = all_paths[:int(len(all_paths)*0.8)] 
# y = all_y[:int(len(all_y)*0.8)] 

# paths_test = all_paths[int(len(all_paths)*0.8):] 
# y_test = all_y[int(len(all_y)*0.8):] 


In [13]:
paths=[]
y=[]

"""Loading all paths and y_labels in df_train_all """
# print("Loading training paths and y values from JSON files")
for index in tqdm(range(np.shape(df_train_all)[0])):
    path, labels = load_data(index, df_train_all[index])
    paths.extend(path)
    y.extend(labels)

print(len(paths) )
print(len(y) )

100%|██████████| 45/45 [01:01<00:00,  1.36s/it]

82549
82549


In [14]:
paths_test=[]
y_test=[]

"""Loading all paths and y_labels in df_train_all """
# print("Loading training paths and y values from JSON files")
for index in tqdm(range(np.shape(df_test_all)[0])):
    path, labels = load_data(index+47, df_test_all[index])
    paths_test.extend(path)
    y_test.extend(labels)

print(len(paths_test) )
print(len(y_test) )


100%|██████████| 3/3 [00:04<00:00,  1.55s/it]

6302
6302


In [15]:
print('There are '+str(y.count(1))+' fake train samples')
print('There are '+str(y.count(0))+' real train samples')
print('There are '+str(y_test.count(1))+' fake train samples')
print('There are '+str(y_test.count(0))+' real train samples')

There are 69229 fake train samples
There are 13320 real train samples
There are 5205 fake train samples
There are 1097 real train samples


In [16]:
"""Underbalancing training """

import random
real=[]
fake=[]
for m,n in zip(paths,y):
    if n==0:
        real.append(m)
    else:
        fake.append(m)
fake=random.sample(fake,len(real))
paths,y=[],[]
for x in real:
    paths.append(x)
    y.append(0)
for x in fake:
    paths.append(x)
    y.append(1)

In [17]:
"""Underbalancing validation/ test"""

import random
real=[]
fake=[]
for m,n in zip(paths_test,y_test):
    if n==0:
        real.append(m)
    else:
        fake.append(m)
fake=random.sample(fake,len(real))
paths_test,y_test=[],[]
for x in real:
    paths_test.append(x)
    y_test.append(0)
for x in fake:
    paths_test.append(x)
    y_test.append(1)

In [18]:
print('There are '+str(y.count(1))+' fake train samples')
print('There are '+str(y.count(0))+' real train samples')
print('There are '+str(y_test.count(1))+' fake train samples')
print('There are '+str(y_test.count(0))+' real train samples')

There are 13320 fake train samples
There are 13320 real train samples
There are 1097 fake train samples
There are 1097 real train samples


In [19]:
paths = np.array(paths)
y = np.array(y)


paths_test = np.array(paths_test)
y_test = np.array(y_test)

In [20]:
y = to_categorical(y, num_classes=2) #convert y training to one hot encodings
y_test = to_categorical(y_test, num_classes=2) #convert y testing to one hot encodings

In [21]:
X=[]
# print("Loading frames training")
for img in tqdm(paths):
    X.append(read_img(img))
    
    
X_test=[]
# print("Loading frames testing")
for img in tqdm(paths_test):
    X_test.append(read_img(img))

100%|██████████| 2194/2194 [00:21<00:00, 104.11it/s]


In [22]:
train_size = len(X)
test_size = len(X_test)

In [23]:
print(y.shape)
print(y_test.shape)


(26640, 2)
(2194, 2)


In [40]:
X_embedded = embed(X)

  0%|          | 37/26640 [00:31<6:17:57,  1.17it/s]


KeyboardInterrupt: 

In [27]:
print(np.shape(X_embedded))

(20, 10, 2304, 5, 5)


In [ ]:
X_test_embedded = embed(X_test)

In [28]:
def window(seq, n):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [29]:
X_rolled = []

for i in range(len(X_embedded)):
        rolled = window(X_embedded[i], n=window_size)
        X_rolled.append(list(rolled))
              

In [ ]:
X_test_rolled = []

for i in range(len(X_test_embedded)):
        rolled = window(X_test_embedded[i], n=window_size)
        X_test_rolled.append(list(rolled)) 

In [32]:
print(np.shape(X_rolled))
print(y.shape)

print(np.shape(X_test_rolled))
print(y_test.shape)

(20, 6, 5, 2304, 5, 5)
(26640, 2)


NameError: name 'X_test_rolled' is not defined

In [34]:
X_rolled = np.reshape(X_rolled, (train_size, supersteps , window_size , 2304* 5* 5))
X_test_rolled = np.reshape(X_test_rolled, (test_size, supersteps , window_size , 1280* 5* 5))


NameError: name 'X_test_rolled' is not defined

In [ ]:
from keras import regularizers


In [35]:
def lstm():
    """Build a simple LSTM network. On the training sample"""
    # Model.
    model = Sequential()
    model.add(TimeDistributed(LSTM(2048, return_sequences=False), input_shape=( supersteps , window_size , 2304* 5* 5 ) ))
    model.add(Dropout(0.5)) #large dropout
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5)) #large dropout
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))

    return model

model = lstm()

optimizer = Adam(lr=1e-5*100, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 6, 2048)           488644608 
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 2048)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 6, 512)            1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 512)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6146      
Total params: 489,699,842
Trainable params: 489,699,842
Non-trainable params: 0
________________________________________

In [37]:
history = model.fit(X_rolled, y, epochs=20, batch_size=supersteps*100, shuffle=True)

# history = model.fit(X_rolled, y, epochs=20, batch_size=supersteps*10, shuffle=True)

model.save_weights("model.h5")


Epoch 1/20
20/20 [==============================] - 4s 221ms/step - loss: 0.8279 - accuracy: 0.5500
Epoch 2/20
20/20 [==============================] - 4s 188ms/step - loss: 1.4901e-09 - accuracy: 1.0000
Epoch 3/20
20/20 [==============================] - 4s 187ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/20
20/20 [==============================] - 4s 188ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/20
20/20 [==============================] - 4s 192ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/20
20/20 [==============================] - 4s 193ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/20
20/20 [==============================] - 4s 191ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/20
20/20 [==============================] - 4s 191ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/20
20/20 [==============================] - 4s 190ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/20
20/20 [==============================] - 4s 191ms/step -

KeyboardInterrupt: 

In [ ]:
y_preds = model.predict_classes(X_test_rolled)


print(y_preds.shape)

In [ ]:
print(y_test.shape)
print(y_preds.shape)

In [ ]:
print("Predictions", y_preds)
print("True Labels", y_test)


In [ ]:
conf_matrix = confusion_matrix(y_test, y_preds)
conf_matrix

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()


print("-------------- Confusion Matrix -------------- ")
print(conf_matrix)


print('True Positives: {}, True Negatives: {}, False Positives: {}, False Negatives: {}'.format(tp, tn, fp, fn))

precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
recall = tp / (tp + fn)
f1 = 2*(precision*recall)/ (precision+recall)
print("-------------- Model Scores -------------- ")
print('Precision: {}, Accuracy: {}, Recall: {}, F1-score: {}'.format(precision, accuracy, recall, f1))

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')


plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('/home/aelbakry1999/Results/accuracy_loss.png')

In [ ]:
df_index = random.randint(0,13)

path, labels = load_data(df_index, df_train_all[df_index])

In [ ]:
Xy_index = random.randint(0,len(path))
test_video_path, test_video_label = path[Xy_index], labels[Xy_index]

In [ ]:
test_video = read_img(test_video_path)

In [ ]:
plt.title(test_video_label)
plt.imshow(test_video[9])

In [ ]:
# test_video_embed = embed([test_video])

In [ ]:
best_model = lstm()
best_model.load_weights("best_model.h5")